In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
dataset = pd.read_csv(r'../dataset/Train.csv')
dataset.head()

,Date,Category,Brand,Day_of_Week,Holiday_Indicator,Past_Purchase_Trends,Price,Discount,Competitor_Price,Sales_Quantity
0,26/09/24,Automotive,BrandA,3,0,17.9,462.30,33.96,359.45,227.0
1,26/09/24,Automotive,BrandA,3,0,66.3,321.28,24.76,49.47,370.0
2,26/09/24,Automotive,BrandA,3,0,14.4,74.85,32.19,245.11,299.0
3,26/09/24,Automotive,BrandA,3,0,34.7,469.12,43.68,144.08,426.0
4,22/09/24,Automotive,BrandA,6,1,93.5,221.76,36.79,478.07,470.0


In [3]:
dataset = dataset.drop(columns='Date')

In [8]:
dataset.isnull().sum().sort_values(ascending=False)

Category                0
Brand                   0
Day_of_Week             0
Holiday_Indicator       0
Past_Purchase_Trends    0
Price                   0
Discount                0
Competitor_Price        0
Sales_Quantity          0
dtype: int64

In [6]:
dataset.describe()

,Day_of_Week,Holiday_Indicator,Past_Purchase_Trends,Price,Discount,Competitor_Price,Sales_Quantity
count,18483.000000,18483.000000,18483.000000,18483.000000,18483.000000,18483.000000,18019.000000
mean,2.991235,0.142022,52.329465,275.111184,27.490937,259.858758,255.665631
std,2.000765,0.349083,27.410577,129.733609,13.022521,126.653165,141.179795
min,0.000000,0.000000,5.000000,50.070000,5.000000,40.070000,10.000000
25%,1.000000,0.000000,28.900000,163.315000,16.265000,149.530000,133.500000
50%,3.000000,0.000000,52.500000,275.150000,27.600000,260.260000,257.000000
75%,5.000000,0.000000,75.700000,387.650000,38.690000,368.520000,377.000000
max,6.000000,1.000000,100.000000,499.900000,49.990000,479.990000,499.000000


In [7]:
dataset['Sales_Quantity'] = dataset['Sales_Quantity'].fillna(dataset['Sales_Quantity'].median())

In [4]:
from sklearn.preprocessing import OneHotEncoder

In [10]:
categoricalColumns = ['Category', 'Brand']

In [11]:
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encodedArray = ohe.fit_transform(dataset[categoricalColumns])

encodedDataset = pd.DataFrame(encodedArray, columns=ohe.get_feature_names_out(categoricalColumns), index=dataset.index)

dataset = dataset.drop(columns=categoricalColumns).join(encodedDataset)

In [12]:
dataset.head()

,Day_of_Week,Holiday_Indicator,Past_Purchase_Trends,Price,Discount,Competitor_Price,Sales_Quantity,Category_Automotive,Category_Beauty,Category_Books,...,Category_Electronics,Category_Home,Category_Sports,Category_Toys,Brand_BrandA,Brand_BrandB,Brand_BrandC,Brand_BrandD,Brand_BrandE,Brand_BrandF
0,3,0,17.9,462.30,33.96,359.45,227.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,3,0,66.3,321.28,24.76,49.47,370.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,3,0,14.4,74.85,32.19,245.11,299.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,3,0,34.7,469.12,43.68,144.08,426.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,6,1,93.5,221.76,36.79,478.07,470.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [13]:
x = dataset.drop(columns='Sales_Quantity')
y = dataset['Sales_Quantity']

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [16]:
from sklearn.ensemble import RandomForestRegressor

In [17]:
rdr = RandomForestRegressor()
rdr.fit(x_train, y_train)

RandomForestRegressor()

In [18]:
rdr.score(x_train, y_train)*100, rdr.score(x_test, y_test)*100

(85.35562709536481, -3.16161744811454)

In [20]:
#important feature selection
impFeatures = pd.Series(rdr.feature_importances_, index=x.columns).sort_values(ascending=False)
impFeatures

Price                   0.188926
Competitor_Price        0.188780
Discount                0.188756
Past_Purchase_Trends    0.180059
Day_of_Week             0.063047
Brand_BrandC            0.013918
Category_Clothing       0.013689
Brand_BrandE            0.013659
Brand_BrandB            0.013633
Category_Beauty         0.013218
Category_Home           0.013178
Brand_BrandF            0.013065
Category_Sports         0.013057
Category_Books          0.012998
Brand_BrandD            0.012971
Category_Automotive     0.012801
Category_Toys           0.012543
Category_Electronics    0.012445
Brand_BrandA            0.012400
Holiday_Indicator       0.006856
dtype: float64